In [15]:
import numpy as np
import pulp

# 样例数据生成
M = 3  # 行数
N = 5  # 列数
I = 4  # w_i 的数量
J = 3  # ww_j 的数量

np.random.seed(1)
w_i = [np.random.randint(0, 2, size=(M, N)) for _ in range(I)]
w_i = [np.ones((M, N)) for _ in range(I)]
ww_j = [np.random.randint(0, 2, size=(M, N)) for _ in range(J)]
ww_j = [np.ones((M, N)) for _ in range(J)]
print(w_i)
print(ww_j)
# 创建优化问题
problem = pulp.LpProblem("Minimize_Sum", pulp.LpMinimize)

# 定义变量
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(M) for j in range(N)), cat="Binary")
y = pulp.LpVariable.dicts("y", (j for j in range(J)), cat="Binary")

# 引入辅助变量 z[j] 用于线性化 y[j] * sum(ww_j[j] * x)
z = pulp.LpVariable.dicts("z", (j for j in range(J)), lowBound=0, cat="Continuous")

# 目标函数：sum(x * w_i) / I
objective = pulp.lpSum(
    sum(w_i[k][i, j] * x[i, j] for k in range(I) for i in range(M) for j in range(N)) / I
)
problem += objective

# 约束 1: z[j] = y[j] * sum(ww_j[j] * x)，并确保 sum(z) <= 0
for j in range(J):
    ww_sum = pulp.lpSum(ww_j[j][i, k] * x[i, k] for i in range(M) for k in range(N))
    problem += z[j] <= ww_sum  # 辅助变量的上界
    problem += z[j] <= y[j] * 1e6  # 辅助变量的条件上界（线性化 y[j] 的影响）
    problem += z[j] >= ww_sum - (1 - y[j]) * 1e6  # 辅助变量的条件下界（线性化 y[j] 的影响）

# 总约束：sum(z) <= 0
problem += pulp.lpSum(z[j] for j in range(J)) <= 0

# 约束 2: sum(x) = 常量 (假设为 K)
K = 1
problem += pulp.lpSum(x[i, j] for i in range(M) for j in range(N)) == K

G = 1
problem += pulp.lpSum(y[i] for i in range(M)) == G
# 求解
problem.solve()

# 输出结果
print("Optimal value:", pulp.value(problem.objective))
print("Optimal x:")
x_result = np.array([[pulp.value(x[i, j]) for j in range(N)] for i in range(M)])
print(x_result)
print("Optimal y:")
y_result = np.array([pulp.value(y[j]) for j in range(J)])
print(y_result)


[array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]]), array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]]), array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]]), array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])]
[array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]]), array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]]), array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])]
Optimal value: 1.0
Optimal x:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
Optimal y:
[1.00000e-06 0.00000e+00 9.99999e-01]
